https://essay.utwente.nl/65559/1/vanderBlonk_MSc_EEMCS.pdf#page55

In [1]:
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting, LagrangesMethod, Lagrangian
from sympy.physics import mechanics
init_vprinting()

import numpy as np
import math
from time import time, sleep

In [2]:
def hat(v):
    a, b, c = v
    return Matrix([[0, -c, b], [c, 0, -a], [-b, a, 0]])

def Ad(H):
    R = H[:3, :3]
    o = H[:3, 3]
    Z = zeros(3)
    ans = [[R, Z], [hat(o) @ R, R]]
    return Matrix(ans)

def Hinv(H):
    R = H[:3, :3]
    p = H[:3, 3]
    z = Matrix(np.zeros((1, 3)))
    return Matrix([[R.T, -R.T @ p], [z, Matrix([1])]])

def jacobian(v, dq):
    n = len(v)
    m = len(dq)
    ans = np.zeros((n, m), dtype=object)
    
    zeros_slist = [(var, 0) for var in dq]
    for i in range(n):
        for j in range(m):
            ans[i, j] = v[i].subs(dq[j], 1).subs(zeros_slist)
    return Matrix(ans)

def partial(arg, x):
    lst = []
    for v in x:
        lst.append(arg.diff(v))
    return Matrix([lst])

In [3]:
Z = Matrix(np.zeros((3,3)))
I = eye(3)
get_w = Matrix([[I, Z]])
get_v = Matrix([[Z, I]])

In [4]:
# Variables
q = Matrix(dynamicsymbols("x y phi_x phi_y phi_z"))
t = symbols("t")
x, y, phix, phiy, phiz = q
dq = q.diff(t)
dx, dy, dphix, dphiy, dphiz = dq
T1, T2, T3 = symbols("Tau_1:4")

if False:
    # Parameters
    l, mb, mh = symbols("l m_b m_h")
    rb, rw = symbols("r_b r_w")
    g = symbols('g')
    alpha = symbols("alpha") # Angle of the omhiwheel point of contact from vertical (45 degrees)
    Ih = diag(*symbols("I_xx I_yy I_zz"))
    Iow, Im, k = symbols("I_ow I_m k")
    
    # Moment of Inertia (placeholder)
    Ib = diag(*([mb*rb**2]*3))
    
else:
#     l = 0.169                      # Distance from ball COM to hat COM (m)
#     mb = 0.624                     # Mass of ball (kg)
#     mh = 2                         # Mass of hat (kg)
#     rb = 0.254 / 2                 # Radius of ball (m)
#     rw = 0.096/2                   # Radius of wheel (m)
#     g = 9.8                        # m/s^2
#     alpha = rad(45)                # Angle of omniwheel axis from the horizontal plane (rad)
#     Ih = diag(1.167e-2, 1.153e-2, 1.234e-2) # Inertia tensor of the hat kg m^2
#     Iow = 1/2*(96e-3)*rw**2        # Inertia of the wheel (scalar) kg m^2
#     Im = 1/2*(0.2)*(32e-3 / 2)**2  # Inertia of the motor winding (scalar) kg m^2 (very rough estimate)
#     k = 13.7                       # Gear ratio of the motor
    
    
    # Paper Parameters
    ms = 3.2
    mow = 0.555
    mgh = 0.16
    mm = 0.24
    rs = 0.115
    wb = 0.2
    h = 0.8
    h1 = 0.66
    h2 = 0.14
    Is = diag(*([2.65e-2]*3))
    Im = 2.42e-6
    Iow = 6.94e-5
    l = 0.405
    g = 9.81
    alpha = math.radians(45)
    beta = 0
    
    mb = 10
    mb1 = 6.2
    mb2 = 0.935
    mb2tot = 3.8
    rw = 0.050
    Ib = diag(8.76e-1, 8.76e-1, 6.67e-2)
    
    # Not specified
    k=10

In [11]:
# Coordinate Transformations
# Frame I: Innertial frame
# Frame 2: Origin at center of the ball
# Frame 5: Origin at COM of the robot body with axes aligned to the robot
HI1 = Matrix([[1, 0, 0, -x],[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
H12 = Matrix([[1, 0, 0, 0], [0, 1, 0, -y], [0, 0, 1, 0], [0, 0, 0, 1]])
H23 = Matrix([[cos(phiz), sin(phiz), 0, 0], [-sin(phiz), cos(phiz), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
H34 = Matrix([[cos(phiy), 0, sin(phiy), 0], [0, 1, 0, 0], [-sin(phiy), 0, cos(phiy), 0], [0, 0, 0, 1]])
H45 = Matrix([[1, 0, 0, 0], [0, cos(phix), -sin(phix), 0], [0, sin(phix), cos(phix), -l], [0, 0, 0, 1]])

# Trig Simplifications
# HI1 = Matrix([[1, 0, 0, -x],[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
# H12 = Matrix([[1, 0, 0, 0], [0, 1, 0, -y], [0, 0, 1, 0], [0, 0, 0, 1]])
# H23 = Matrix([[1, phiz, 0, 0], [-phiz, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
# H34 = Matrix([[1, 0, phiy, 0], [0, 1, 0, 0], [-phiy, 0, 1, 0], [0, 0, 0, 1]])
# H45 = Matrix([[1, 0, 0, 0], [0, 1, -phix, 0], [0, phix, 1, -l], [0, 0, 0, 1]])


HI2 = HI1 @ H12
HI3 = HI2 @ H23
HI4 = HI3 @ H34
HI5 = HI4 @ H45

In [12]:
T2II = Matrix([0, 0, 0, dx, dy, 0])
T22I = Ad(HI2) @ T2II

v22I = get_v @ T22I

T25I = Ad(HI5) @ T2II
T25I

⎡                                   0                                   ⎤
⎢                                                                       ⎥
⎢                                   0                                   ⎥
⎢                                                                       ⎥
⎢                                   0                                   ⎥
⎢                                                                       ⎥
⎢(sin(φₓ)⋅sin(φ_y)⋅cos(φ_z) + sin(φ_z)⋅cos(φₓ))⋅ẏ + cos(φ_y)⋅cos(φ_z)⋅ẋ ⎥
⎢                                                                       ⎥
⎢(-sin(φₓ)⋅sin(φ_y)⋅sin(φ_z) + cos(φₓ)⋅cos(φ_z))⋅ẏ - sin(φ_z)⋅cos(φ_y)⋅ẋ⎥
⎢                                                                       ⎥
⎣                    sin(φₓ)⋅cos(φ_y)⋅ẏ - sin(φ_y)⋅ẋ                    ⎦

In [13]:
wS2I = Matrix([dy/rs, -dx/rs, 0])

In [63]:
# Body Velocities
T1II = Matrix([0, 0, 0, 1, 0, 0])
T211 = Matrix([0, 0, 0, 0, 1, 0])
T322 = Matrix([0, 0, 1, 0, 0, 0])
T433 = Matrix([0, 1, 0, l, 0, 0])
T544 = Matrix([-1, 0, 0, 0, l, 0])
J = Matrix([[T1II, Ad(Hinv(HI1))@T211, Ad(Hinv(HI2))@T322, Ad(Hinv(HI3))@T433, Ad(Hinv(HI4))@T544]])

qJdot = Matrix([dx, dy, dphiz, dphiy, dphix])

T5II = J @ qJdot # Body Twist
T55I = Ad(HI5) @ T5II

In [68]:
T5II

⎡                                                                                        -sin(φ_z)
⎢                                                                                                 
⎢                                                                                             sin(
⎢                                                                                                 
⎢                                                                                             sin(
⎢                                                                                                 
⎢                        ((x⋅sin(φ_z) + y⋅cos(φ_z))⋅sin(φ_y)⋅cos(φ_z) + (-x⋅sin(φ_y)⋅cos(φ_z) + y⋅
⎢                                                                                                 
⎢               ((x⋅sin(φ_y)⋅cos(φ_z) - y⋅sin(φ_y)⋅sin(φ_z))⋅cos(φ_y)⋅cos(φ_z) + (-x⋅cos(φ_y)⋅cos(
⎢                                                                                                 
⎣(-(-x⋅sin

In [69]:
# Omni Wheel Velocities
# Wheel 1 is on the positive x axis, with the other second wheel at +120 degrees and the third wheel at +240 degrees

H25 = H23 @ H34 @ H45
R25 = H25[:3, :3]

w55I = get_w @ T55I # angular velocity of Frame 5 wrt Frame I expressed in Frame 5

wS55 = R25 @ wS2I - w55I # wS55 is the angular velocity of the ball with respect to the body frame

rw1 = rs * Matrix([sin(alpha), 0, cos(alpha)])
rw2 = rs * Matrix([-sin(alpha)/2, sqrt(3)/2*sin(alpha), cos(alpha)])
rw3 = rs * Matrix([-sin(alpha)/2, -sqrt(3)/2*sin(alpha), cos(alpha)])

half = Rational(1, 2)
uw1 = Matrix([0, 1, 0])
uw2 = Matrix([-sqrt(3)/2, -half, 0])
uw3 = Matrix([sqrt(3)/2, -half, 0])

sw1 = (hat(wS55)@rw1).T @ uw1 / rw
sw2 = (hat(wS55)@rw2).T @ uw2 / rw
sw3 = (hat(wS55)@rw3).T @ uw3 / rw

In [70]:
# Ball Energies
RI2 = HI2[:3, :3]
H2I = Hinv(HI2)
wSII = get_w @ Ad(H2I) @ Matrix([wS2I, v22I])
Ts = T22I.T @ diag(Is, ms*I) @ T22I / 2 + wS2I.T@Is@wS2I / 2
Vs = 0

In [71]:
# Body Energies
R5I = Hinv(HI5)[:3, :3]
Tb = T55I.T @ diag(Ib, ms*eye(3)) @ T55I / 2
Vb = mb * Matrix([0,0,g]).T @ R5I @ Matrix([0, 0, l])

In [72]:
# Omniwheel Energies
Tws = (Iow + Im*k**2)/2 * (sw1**2 + sw2**2 + sw3**2)
Vws = 0

In [73]:
# Lagrangian
L = Ts[0] + Tb[0] + Tws[0] - (Vs + Vb[0] + Vws)

In [74]:
# Torque Transformations
jac = jacobian(Matrix([sw1, sw2, sw3]), dq) # from sw1, sw2, sw3
T_ext = jac.T @ Matrix([T1, T2, T3])

In [75]:
t0 = time()
LM = LagrangesMethod(L, q)
_ = LM.form_lagranges_equations()
print("Took %.1f seconds"%(time() - t0))

Took 35.0 seconds


# Linearization

In [76]:
xvar = Matrix([q, dq])
u = Matrix([T1, T2, T3])

xbar_slist = [(v, 0) for v in xvar] + [(v, 0) for v in u]

In [77]:
Mlin = LM.mass_matrix.subs(xbar_slist)
Mlin = np.array(Mlin, dtype=float).round(5)
Minv = np.linalg.inv(Mlin).round(3)

In [78]:
Mlin * dq.diff(t)

⎡2.60274⋅φ_̈y + 8.4972⋅ẍ ⎤
⎢                       ⎥
⎢-0.01074⋅φₓ̈ + 8.4972⋅ÿ ⎥
⎢                       ⎥
⎢0.87724⋅φₓ̈ - 0.01074⋅ÿ ⎥
⎢                       ⎥
⎢2.97676⋅φ_̈y + 2.60274⋅ẍ⎥
⎢                       ⎥
⎣      0.06917⋅φ_̈z      ⎦

In [79]:
t0 = time()
dG = np.array(partial(LM.forcing, xvar).subs(xbar_slist).doit(), dtype=float).round(5)
print("Took %f.2 seconds" %(time()-t0))

Took 137.855292.2 seconds


In [80]:
Eq(Mlin @ dq.diff(t), Matrix(dG.round(2))[:,:5] @ q)

⎡2.60274⋅φ_̈y + 8.4972⋅ẍ ⎤   ⎡    0    ⎤
⎢                       ⎥   ⎢         ⎥
⎢-0.01074⋅φₓ̈ + 8.4972⋅ÿ ⎥   ⎢    0    ⎥
⎢                       ⎥   ⎢         ⎥
⎢0.87724⋅φₓ̈ - 0.01074⋅ÿ ⎥ = ⎢39.73⋅φₓ ⎥
⎢                       ⎥   ⎢         ⎥
⎢2.97676⋅φ_̈y + 2.60274⋅ẍ⎥   ⎢39.73⋅φ_y⎥
⎢                       ⎥   ⎢         ⎥
⎣      0.06917⋅φ_̈z      ⎦   ⎣    0    ⎦

In [81]:
Mlin

array([[ 8.4972 ,  0.     ,  0.     ,  2.60274,  0.     ],
       [ 0.     ,  8.4972 , -0.01074,  0.     ,  0.     ],
       [ 0.     , -0.01074,  0.87724,  0.     , -0.     ],
       [ 2.60274,  0.     ,  0.     ,  2.97676,  0.     ],
       [ 0.     ,  0.     , -0.     ,  0.     ,  0.06917]])

In [82]:
Abot = Minv @ -dG

In [83]:
dT_ext = np.array(partial(T_ext, u).subs(xbar_slist).doit(), dtype=float).round(5)
Bbot = Minv @ dT_ext

In [84]:
Matrix(Abot.round(2)[:,:5])

⎡0  0    0      5.6    0⎤
⎢                       ⎥
⎢0  0  -0.04     0     0⎥
⎢                       ⎥
⎢0  0  -45.29    0     0⎥
⎢                       ⎥
⎢0  0    0     -18.24  0⎥
⎢                       ⎥
⎣0  0    0       0     0⎦

In [85]:
Matrix(Bbot.round(2))

⎡  0      1.77   -1.77 ⎤
⎢                      ⎥
⎢-1.67    0.83    0.83 ⎥
⎢                      ⎥
⎢ 1.84   -0.92   -0.92 ⎥
⎢                      ⎥
⎢  0     -1.08    1.08 ⎥
⎢                      ⎥
⎣-23.51  -23.51  -23.51⎦